In [2]:
import joblib
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.decomposition import PCA
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
class BestTimeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, col="best_time"):
        self.col = col
        self.mlb = MultiLabelBinarizer(classes=list(range(1, 13)))
        self.pca = PCA(n_components=1)

    def fit(self, X, y=None):
        X = X.copy()
        month_lists = [self._months_to_list(s) for s in X[self.col]]
        one_hot = self.mlb.fit_transform(month_lists)
        quarters = self._months_to_quarters(one_hot)
        self.pca.fit(quarters)
        return self

    def transform(self, X):
        X = X.copy()
        month_lists = [self._months_to_list(s) for s in X[self.col]]
        one_hot = self.mlb.transform(month_lists)
        quarters = self._months_to_quarters(one_hot)
        X[self.col] = self.pca.transform(quarters)
        return X

    def _months_to_list(self, s):
        mapping = {"Jan":1, "Feb":2, "Mar":3, "Apr":4, "May":5, "Jun":6,
                   "Jul":7, "Aug":8, "Sep":9, "Oct":10, "Nov":11, "Dec":12}
        return [mapping[m.strip()] for m in s.split(",") if m.strip() in mapping]

    def _months_to_quarters(self, one_hot):
        Q1 = one_hot[:, [0,1,2]].sum(axis=1)
        Q2 = one_hot[:, [3,4,5]].sum(axis=1)
        Q3 = one_hot[:, [6,7,8]].sum(axis=1)
        Q4 = one_hot[:, [9,10,11]].sum(axis=1)
        return np.vstack([Q1, Q2, Q3, Q4]).T

In [4]:
country_transformer = joblib.load("country_transformer.pkl")
time_transformer = joblib.load("time_transformer.pkl")
scaler = joblib.load("scaler.pkl")
kmeans = joblib.load("kmeans.pkl")
results_df = joblib.load("results_df.pkl")
glove_model = KeyedVectors.load("glove_vectors.gensim", mmap='r')

In [5]:
def tokens_to_glove_vector(tokens, model, vector_size=50):
    """Convert list of tokens to average GloVe vector."""
    vectors = [model[word] for word in tokens if word in model]
    if not vectors:
        return np.zeros(vector_size)
    return np.mean(vectors, axis=0)

In [6]:
def predict_destination(city, country, months, category_tokens,
                        country_transformer, time_transformer,
                        glove_model, scaler, kmeans, results_df):
    

    # 1. Country PCA transform → shape (1, 5)
    country_vec = country_transformer.transform(
        pd.DataFrame({"Country": [country]})
    )

    # 2. Time PCA transform → shape (1, 1)
    time_vec = time_transformer.transform(
        pd.DataFrame({"Best_Time_to_Travel": [months]})
    )

    # 3. GloVe category → shape (1, 50)
    glove_vec = tokens_to_glove_vector(category_tokens, glove_model, 50).reshape(1, -1)

    # 4. Combine into one row → shape (1, 56)
    full_features = np.hstack([time_vec,country_vec,glove_vec])

    # 5. Scale and predict cluster
    scaled = scaler.transform(full_features)
    cluster = int(kmeans.predict(scaled)[0])

    # 6. Find similar cities in results_df
    matches = results_df[results_df["Cluster"] == cluster][["City", "Category"]].reset_index(drop=True)

    return {
        "predicted_cluster": cluster,
        "similar_cities": matches
    }

In [7]:
result = predict_destination(
    city="Kyoto",
    country="Japan",
    months="Mar, Apr, May, Oct, Nov",
    category_tokens=[ "food"],
    country_transformer=country_transformer,
    time_transformer=time_transformer,
    glove_model=glove_model,
    scaler=scaler,
    kmeans=kmeans,
    results_df=results_df
)

print(result)


c:\Users\OMEN\anaconda3\envs\gensim-env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\OMEN\anaconda3\envs\gensim-env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


{'predicted_cluster': 3, 'similar_cities':          City                                           Category
0       Tokyo  culture, food, technology, shopping, temples, ...
1       Seoul  culture, food, K-pop, shopping, history, palac...
2   Hong Kong  skyline, shopping, food, dim sum, hiking, fina...
3     Bangkok  culture, temples, food, street markets, nightl...
4   Singapore  food, shopping, gardens, modern architecture, ...
5    Istanbul  history, culture, mosques, bazaars, food, arch...
6       Osaka  food (street food), nightlife, modern culture,...
7       Busan  beach, port city, seafood markets, temples, cu...
8      Taipei  food (night markets), technology, culture, tem...
9      Manila  history (Intramuros), culture, food, gateway t...
10      Seoul  culture, food, K-pop, shopping, history, palac...
11  Singapore  food, shopping, gardens, modern architecture, ...
12     Indore       food (street food capital), history, palaces}
